# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных:


- `children` — количество детей в семье

- `days_employed` — общий трудовой стаж в днях

- `dob_years` — возраст клиента в годах

- `education` — уровень образования клиента

- `education_id` — идентификатор уровня образования

- `family_status` — семейное положение

- `family_status_id` — идентификатор семейного положения

- `gender` — пол клиента

- `income_type` — тип занятости

- `debt` — имел ли задолженность по возврату кредитов

- `total_income` — ежемесячный доход

- `purpose` — цель получения кредита

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

# Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('D:/repository/projects/1_предобработка данных/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Используя метод `info` проверю типы данных и общую информацию

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Используя метод `isna` проверю наличие пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Методом `describe` получу основные статистические данные

In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### В каждом из столбцов посмотрю на уникальные  значения

In [6]:
columns = data.columns

for column in columns:
    print(column, end='\n\n')
    print(data[column].value_counts().sort_index(), end='\n\n\n')

children

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64


days_employed

-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: days_employed, Length: 19351, dtype: int64


dob_years

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      

Таблица состоит из 12 столбцов, имеется 21525 записей.  Так же в представленной таблице увидел, что в столбце `days_employed` есть отрицательные значения и слишком большие, в столбце `purpose` одни и те же категории названы по разному, так же как и в столбце `education` и `purpose`


Столбцы `days_employed` и `total_income` имеет тип float64, считаю что стаж и доход должны быть целыми числоми.
так же в этих столбцах есть пропущенные значения


- Cтолбец `children`: есть отрицательные значения -1, а максимальное значение по `children` - 20 (выглядит как ошибка выгрузки, значений не смого, поэтому заменим 0). 

- Столбец `days_employed` - 173 года (63046 дней) - явно не корректное значение. Так же имеется 15906 отрицательных значений, буду рассматривать значения по модулю.

- В столбце `dob_years` минимальное значение равено 0

- В столбце `education` есть одни и те же категории в разном регистре – сделаю всё одного вида и избавлюсь от дубликатов.

- В столбце `purpose` та же картина, что и в `education`, думаю целесообразно будет применить лемматизацию.

# Шаг 2. Предобработка данных

### Обработка пропусков

Заполню пропуски в столбце `days_employed`. Для этого добавлю новый столбец `ratio_days_employed`, чтобы впоследствии заполнить NaN в столбце `days_employed` с учетом среднего этого показателя по группе. в расчётах учичтывал, что трудоустроиться возможно по достижении 18 лет

Избавлюсь от отрицательных значений и заменю значения NaN в датафрейме 

In [7]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['days_employed'] = data['days_employed'].fillna(value = data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(value = data['total_income'].median())
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Удалю неверные значения в столбце `children`, оставлю значения от 0 до 5 детей

In [8]:
data = data[(data['children'] >= 0) & (data['children'] <= 5)]
data.sort_values('children')['children'].value_counts().sort_index()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Проверю остались ли пропуски

In [9]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В результате видим, что пропусков не осталось. Т.е. имиются полные данные, которые необходимы для последующего анализа.

### Замена типа данных

В столбцах `days_empoyed` и `total_income`' заменю тип данных на integer

In [10]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

**чтобы перейти к поиску дубликатов необходимо привести таблицу в читаемый вид, а именно уровень образования клиента к  единому регистру**

In [11]:
data['education'] = data['education'].str.lower()

Узнаю сколько дубликатов имеется в датайрейме

In [12]:
data.duplicated().sum()

71

Рассмотрю какие именно это за дубликакты

In [13]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,2194,46,среднее,1,женат / замужем,0,F,сотрудник,0,145017,высшее образование
520,0,2194,35,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
541,0,2194,57,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
554,0,2194,60,среднее,1,женат / замужем,0,M,сотрудник,0,145017,покупка недвижимости
680,1,2194,30,высшее,0,женат / замужем,0,F,госслужащий,0,145017,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,2194,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,2194,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


В таблице видно, что это и не дубликаты вовсе, т.к. совпадает всего один столбец `total_income`. Следовательнно делаю вывод, что ничего не повлияет на правильность конечного результата, по крайней мере на этом этапе.

### Лемматизация

В столбце `purpose` хранятся цели получения кредита, многие из них одинаковые,  но описаны по разному. Вычислю количество уникальных лемм.

In [14]:
# ещё раз посмотрю список
data['purpose'].value_counts().sort_values(ascending=False)

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

In [15]:
# найду самые частоиспользуемые слова, сразу откинув пробелы и предлоги 
from collections import Counter
m = Mystem()
purposes = []

for word in data['purpose']:
    lemma = m.lemmatize(word)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)

print(Counter(purposes))

Counter({'недвижимость': 6330, 'покупка': 5880, 'жилье': 4450, 'автомобиль': 4288, 'образование': 3997, 'операция': 2593, 'свадьба': 2337, 'свой': 2224, 'строительство': 1870, 'высокий': 1368, 'получение': 1311, 'коммерческий': 1307, 'для': 1291, 'жилой': 1225, 'сделка': 939, 'заниматься': 908, 'дополнительный': 902, 'проведение': 772, 'сыграть': 769, 'сдача': 651, 'семья': 640, 'собственный': 628, 'ремонт': 609, 'подержанный': 484, 'подержать': 472, 'приобретение': 460, 'профильный': 432})


In [16]:
# в итоговм списке целей оставлю только наиболее частовстречаемые категории
purpose_category = [
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование',
    'свадьба'
]

По результатам лемматизации выявил 5 категорий по которым буду проводить категоризацию, с целью дальнейшего анализа и составления выводов

### Категоризация данных

In [17]:
def category_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for category in purpose_category:
        for lemma in lemmas:
            if category in lemma:
                return category

In [18]:
# к датафрейму добавлю новый столбецс с категориями, использовав метод apply
data['purpose_category'] = data['purpose'].apply(category_purpose)
# проверю добавился ли столбец в татблицу и корректно ли он работает
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [19]:
# далее разделю заёмщиков на категории по количеству детей
def category_children(children):
    if children == 0:
        return 'нет детей'
    if children <=2:
        return 'среднестатистические'
    if children >= 3:
        return 'многодетные'

In [20]:
# добавлю ещё один столбец, с категориями по количеству детей  
data['children_category'] = data['children'].apply(category_children)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей


In [21]:
# добавлю столбец, с категориями по семейному положению, т.е. сокращу количество категорий, для более удобного анализа
def category_family_status(family_status):
    if family_status == 'женат / замужем':
        return 'в браке'
    if family_status == 'гражданский брак':
        return 'в браке'
    
    return 'не в браке'

In [22]:
data['family_status_category'] = data['family_status'].apply(category_family_status)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,в браке


In [23]:
# проведу категоризацию по уровню дохода
def category_total_income(total_income):
    if total_income < 50000:
        return 'низкий'
    elif total_income <= 350000:
        return 'средний'
    elif total_income < 350000:
        return 'выше среднего'
    else:
        return 'высокий'

In [24]:
# добавлю этот столбец в датафрейм
data['total_income_category'] = data['total_income'].apply(category_total_income)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические,в браке,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические,в браке,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей,в браке,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,в браке,средний
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,в браке,средний


Таблица готова к анализу, категоризация произведена. Можно приступать к ответам на вопросы

# Шаг 3. Анализ завивсимостей

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [25]:
# составлю сводную таблицу
relation_children = data.groupby('children_category')['debt'].agg(['mean', 'sum', 'count'])
format_dict = { 'mean': '{:.2%}' }
relation_children.style.format(format_dict)

,mean,sum,count
children_category,,,
многодетные,8.16%,31,380
нет детей,7.51%,1063,14149
среднестатистические,9.28%,638,6873


Бездетные чаще выплачивают кредит в срок, считаю что это не удивительно, гораздо проще планировать бюджет, когда нет детей,так же видно что это самая большая категория. В среднестатистических семьях, где 1-2 детей, отстают почти на 2 процента, хотя обращений в банк от этой категории в 2 раза ниже. Ну и в заключении многодетные семьи, их достаточно мало, но тем не менее ситуация по возвратам всё равно хуже, нежели у бездетных. Следовательно можно сделать вывод, что основными клиентами банка являются люди без детей и же лучше всего справляются с выплатами кредита в срок.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [26]:
relation_family_status = data.groupby('family_status_category')['debt'].agg(['mean', 'sum', 'count'])
format_dict = { 'mean': '{:.2%}' }
relation_family_status.style.format(format_dict)

,mean,sum,count
family_status_category,,,
в браке,7.97%,1312,16462
не в браке,8.50%,420,4940


Клиенты находящщиеся в браке чаще выплачивают кредит в срок и в то же время примерно в 3 раза чаще обращаются для получения кредита.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [27]:
relation_total_income = data.groupby('total_income_category')['debt'].agg(['mean', 'sum', 'count'])
format_dict = { 'mean': '{:.2%}' }
relation_total_income.style.format(format_dict)

,mean,sum,count
total_income_category,,,
высокий,6.43%,55,856
низкий,6.20%,23,371
средний,8.20%,1654,20175


Чащё всего за кредитом обращаются люди со средним заработком и в то же время они чаще не могут выплатить его в срок. Интересным для меня оказалось, что люди с низким уровнем дохода реже всего уходят в 'просрочку', видимо небольшой доход заставляет бережнее относиться к деньгам. Люди с высоким уровнем дохода почти не отстают от людей с низким, видимо так же осознают ценность денег.

- **Как разные цели кредита влияют на его возврат в срок?**

In [28]:
relation_purpose = data.groupby('purpose_category')['debt'].agg(['mean', 'sum', 'count'])
format_dict = { 'mean': '{:.2%}' }
relation_purpose.style.format(format_dict)

,mean,sum,count
purpose_category,,,
автомобиль,9.33%,400,4288
жилье,6.92%,308,4450
недвижимость,7.46%,472,6330
образование,9.23%,369,3997
свадьба,7.83%,183,2337


Как видно в таблице, те кто хотят купить жильё или др недвижимость более отвественно относятся к выплатам, они же ии чаще других обращаются за кредитом. Самая плохая картина у тех, кто хочет автомобиль и учиться, эти категория не сильно отстаёт по количеству заёмщиков от тех, кто покупает квартиру

# Вывод


По результатам исследования можно сказать, что что самые ответственные заёмщики имеют низкий или высокий заработок, видимо эти категории понастоящему осознают ценность денег. Большинство из них находятся в браке и не имеют детей, за то имеют большие амбиции, сначала хотят обеспечить себя жильём (основная цель получения ссуды), а уже в дальнейшем, возможно, завести детей.